In [141]:
import numpy as np
import pandas as pd
import math
import json

In [114]:
def find_ch(str, ch):
    for i, ltr in enumerate(str):
        if ltr == ch:
            return i

In [115]:
ref = pd.read_csv('subject_code_to_name.csv')

In [116]:
sorted_data = {'A': {}, 'B': {}, 'C': {}, 'D': {}, 'E': {}, 'F': {}, 'G': {}, 'Other': {}}
keys = []

In [117]:
# sorts all the data into a suitable data structure
for i, code in enumerate(ref['code']):
    subject_group = code[1]
    if code[1] == 'M' or code[0] == '4':
        subject_group = 'Other'
    
    if code[0] == 'E':
        subject_group = 'E'
        code = '3' + code
    
    keys.append(code)

    sorted_data[subject_group][code] = [ref['Name'][i]]

In [118]:
timetable = pd.read_csv('timetable.csv')
print(timetable)

              Date        Time  Room  Code
0    Mon 20 Apr am  9.30-11.10   LT1   4F8
1              NaN         NaN   NaN   3G5
2              NaN  9.30-11.10   LT2   3B5
3              NaN         NaN   NaN  4A12
4              NaN  9.30-12.40   LR3   3A1
..             ...         ...   ...   ...
125            NaN         NaN   LT2   3G1
126            NaN         NaN   NaN   4C4
127            NaN         NaN   LR3   3A5
128            NaN         NaN  LR3B  4B19
129            NaN         NaN   NaN  4F12

[130 rows x 4 columns]


In [119]:
for i, code in enumerate(timetable['Code']):
    testcode = code
    try: 
        if code[0] == 'E':
            testcode = '3' + str(testcode)
    except:
        pass
        
        
    # correct 3rd year subject
    if testcode in keys:
        datetime = ''
        
        for j in range(1000):
            if type(timetable['Date'][i-j]) == str:
                # now we have correct time / date info
                # just need to format it into the correct string config for js
                start_time = ['9:30:00', '14:00:00']
                raw = timetable['Date'][i-j]
                space = []
                
                for i, char in enumerate(raw):
                    if char == ' ':
                        space.append(i)
                        
                month = raw[space[1]+1:space[2]]
                day = raw[space[0]+1:space[1]]
                
                if month == "April":
                    month = "Apr"
                    
                datetime = datetime + month + ' ' + day + ',  2020 '
                if raw[space[2]+1:] == 'am':
                    datetime = datetime + start_time[0]
                else: 
                    datetime = datetime + start_time[1]
                
                break
                
        subject_group = testcode[1]
        if subject_group == 'M' or testcode[0] == '4':
            subject_group = 'Other'
            
        if len(sorted_data[subject_group][testcode]) == 1:
            sorted_data[subject_group][testcode].append(datetime)
        
print(sorted_data)
                
#format I want in the end: "Jun 3,  2020 9:00:00"
#we get                  : :Mon 20 Apr am:

{'A': {'3A1': ['Fluid Mechanics I', 'Apr 20,  2020 9:30:00'], '3A3': ['Fluid Mechanics II', 'Apr 29,  2020 9:30:00'], '3A5': ['Thermodynamics and Power Generation', 'May 1,  2020 14:00:00'], '3A6': ['Heat and Mass Transfer', 'Apr 24,  2020 9:30:00']}, 'B': {'3B1': ['Radio Frequency Electronics', 'Apr 23,  2020 14:00:00'], '3B2': ['Integrated Digital Electronics', 'Apr 24,  2020 9:30:00'], '3B3': ['Switch-Mode Electronics', 'Apr 28,  2020 14:00:00'], '3B4': ['Electric Drive Systems', 'Apr 20,  2020 14:00:00'], '3B5': ['Semiconductor Engineering', 'Apr 20,  2020 9:30:00'], '3B6': ['Photonic Technology', 'Apr 27,  2020 9:30:00']}, 'C': {'3C1': ['Materials Processing and Design (Engineering)', 'Apr 23,  2020 9:30:00'], '3C5': ['Dynamics', 'Apr 21,  2020 14:00:00'], '3C6': ['Vibration', 'Apr 28,  2020 9:30:00'], '3C7': ['Mechanics of Solids', 'Apr 21,  2020 9:30:00'], '3C8': ['Machine Design', 'Apr 23,  2020 14:00:00'], '3C9': ['Fracture Mechanics of Materials and Structures', 'Apr 29,  202

In [143]:
# save as json files for each subject group
file = open("../templates/data/dates.js","w+")
for key in sorted_data.keys():
    data = "var " + key + "_raw ='" + json.dumps(sorted_data[key]) + "';" + "\n"
    file.write(data)    
    parsedata = "var " + key + " = JSON.parse(" + key + "_raw);\n\n"
    file.write(parsedata)
file.close()

In [147]:
with open('alldata.json', 'w') as outfile:
    json.dump(sorted_data, outfile)